Default imports

In [19]:
import os
import chardet
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.feature_selection import chi2
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import projections


Crawling and fetching data from external website

In [32]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

# Max = 2022, Min = 1972 (Last 50 years)
crawl_url = "https://www.ngdc.noaa.gov/hazel/view/hazards/tsunami/event-data?maxYear=2022&minYear=1972"

browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
try:
  browser.get(crawl_url)
  timeout = 15
  WebDriverWait(browser, timeout).until(ec.presence_of_element_located((By.CLASS_NAME, 'haz-grid')))
  i = 0
  for i in range(10):
    browser.execute_script(
        "document.querySelector('#maincontent > div > div.haz-data-grid-container > div > div > div > div.haz-data-grid-container > div > div:nth-child(2) > div > div:nth-child(1) > div').scrollLeft += 350;")
    i += 1
  html = browser.page_source
  crawl_soup = BeautifulSoup(html, 'html.parser')
except TimeoutException:
  print("Can't handle...")

titles = []
rows = []

# The page load in a way that I need to scroll to the right to see whole table,
# Couldn't make it to work, so solution to crawl and click on download TSV file

soup_rows = crawl_soup.find('div', {'class': 'haz-grid'})
children = soup_rows.findAll('div', recursive=False)
inner = children[1].find('div', recursive=False)
sec_inner = inner.findAll('div', recursive=False)
soup_column_names_e = sec_inner[0].find('div', recursive=False)
soup_column_names_e2 = soup_column_names_e.find('div', recursive=False)
soup_column_names_div = soup_column_names_e2.findAll('div', recursive=False)
soup_data_rows = sec_inner[1]

for div in soup_column_names_div:
  inner_div = div.find('div', recursive=False)
  second_div = inner_div.find('div', recursive=False)
  print(second_div.text)

browser.quit()




====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/orrgoren/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


Total Deaths
Total Death Description
Total Missing
Total Missing Description
Total Injuries
Total Injuries Description
Total Damage ($Mil)
Total Damage Description
Total Houses Destroyed
Total Houses Destroyed Description


Cleaning The Data recieved

In [29]:
tsv_file = ''
for file in os.listdir('./'):
  if file.endswith('.tsv'):
    tsv_file = file

# with open(file, encoding='ISO-8859-1') as fp:
#   result = chardet.detect(fp.read(100000))
#   fp.close()

# print(result)

df = pd.read_table(file, sep='\t')
df

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcf in position 0: invalid continuation byte